# Treinamento do Modelo: Curador.IA

**Contexto:** Projeto da disciplina de Processamento Digital de Imagens (PDI) / Visão Computacional.
**Objetivo:** Treinar uma Rede Neural Convolucional (CNN) capaz de classificar obras de arte de 5 mestres da pintura (Monet, Da Vinci, Picasso, Dalí, Van Gogh).

### Pipeline deste Notebook:

1.  **Transfer Learning:** Utilizamos a arquitetura **MobileNetV2** (pré-treinada no ImageNet) como extrator de características, adicionando camadas densas personalizadas para nossa classificação.
2.  **Robustez (Data Augmentation):** O diferencial deste treino é a configuração agressiva do `ImageDataGenerator`. Simulamos variações de **brilho, rotação e perspectiva** para garantir que o modelo funcione ao capturar fotos de **telas de computador e impressões**, ignorando reflexos e distorções.
3.  **Output:** O código gera o arquivo `modelo_artes.h5`, que será utilizado no frontend (Streamlit).

---

In [ ]:
import os
import zipfile
from google.colab import drive

# 1. Montar o Google Drive
drive.mount('/content/drive')

# 2. Configurar caminhos
caminho_zip = '/content/drive/My Drive/dataset.zip'
caminho_extracao = '/content/dataset_temp'

# 3. Extrair
if os.path.exists(caminho_zip):
    print("Extraindo o dataset")
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(caminho_extracao)
    print("Sucesso! pasta extraída.")
else:
    print("ERRO: Não encontrei a pasta no seu Drive.")

Mounted at /content/drive
A extrair o dataset (isto pode demorar alguns segundos)...
✅ Sucesso! Ficheiros extraídos.


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,             # Normalizar cores
    rotation_range=20,          # Roda a imagem
    width_shift_range=0.15,     # Move a imagem para os lados
    height_shift_range=0.15,    # Move a imagem para cima/baixo
    shear_range=0.15,           # Deforma a perspetiva
    zoom_range=0.25,            # Simula zoom ou estar muito perto/longe
    brightness_range=[0.5, 1.5],# Simula ecrãs muito brilhantes ou escuros
    channel_shift_range=30.0,   # Altera ligeiramente as cores
    horizontal_flip=True,       # Espelha a imagem
    fill_mode='nearest'
)

# Para a validação, usamos apenas a normalização
val_datagen = ImageDataGenerator(rescale=1./255)

# Carregar as imagens das pastas
print("A carregar imagens de Treino...")
train_generator = train_datagen.flow_from_directory(
    f'{caminho_extracao}/dataset_projeto/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

print("A carregar imagens de Validação...")
validation_generator = val_datagen.flow_from_directory(
    f'{caminho_extracao}/dataset_projeto/validation',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

print(list(train_generator.class_indices.keys()))

A carregar imagens de Treino...
Found 923 images belonging to 5 classes.
A carregar imagens de Validação...
Found 232 images belonging to 5 classes.

⚠️ ORDEM DAS CLASSES (Copie isto para o seu código final):
['claude_monet', 'leonardo_da_vinci', 'pablo_picasso', 'salvador_dali', 'vincent_van_gogh']


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# 1. Carregar a base pré-treinada
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelamos a base
base_model.trainable = False

# 2. Adicionar as nossas camadas personalizadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x) 
x = Dropout(0.3)(x)                  # Ajuda a evitar decorar (overfitting)
predictions = Dense(5, activation='softmax')(x) # 5 Classes de Artistas

model = Model(inputs=base_model.input, outputs=predictions)

# 3. Compilar
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 4. Treinar
history = model.fit(
    train_generator,
    epochs=12,
    validation_data=validation_generator
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
A iniciar treino... (Isto vai demorar cerca de 5 a 10 minutos)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/12
29/29 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - accuracy: 0.4823 - loss: 1.3968 - val_accuracy: 0.6897 - val_loss: 0.7897
Epoch 2/12
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 662ms/step - accuracy: 0.6714 - loss: 0.8712 - val_accuracy: 0.7241 - val_loss: 0.6888
Epoch 3/12
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 692ms/step - accuracy: 0.7136 - loss: 0.7370 - val_accuracy: 0.7543 - val_loss: 0.6378
Epoch 4/12
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 657ms/step - accuracy: 0.7524 - loss: 0.6372 - val_accuracy: 0.7112 - val_loss: 0.6945
Epoch 5/12
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 703ms/step - accuracy: 0.7672 - loss: 0.6396 - val_accuracy: 0.7586 - val_loss: 0.5721
Epoch 6/12
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 667ms/step - accuracy: 0.8086 - loss: 0.5388 - val_accuracy: 0.7672 - val_loss: 0.5739
Epoch 7/12
29/29 ━━━━━━━━━━━━━━━━━━━━ 20s 702ms/step - accuracy: 0.7814 - loss: 0.5379 - val_accuracy: 0.7543 - val_loss: 0.6112
Epoch 8/12
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 664ms/step - accuracy: 0.8462 - loss: 0.4237 - val_accurac

In [ ]:
#Salvar e baixar o arquivo
from google.colab import files

nome_modelo = 'modelo_artes.h5'
model.save(nome_modelo)

files.download(nome_modelo)

Modelo modelo_artes_v2.h5 guardado!
A iniciar download para o seu computador...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>